In [1]:
import numpy as np
import pandas as pd
import sys

In [2]:
sys.path.append('../../nuclearpy_models/')

In [3]:
from models.BE import dz_be, semf_be
from models.BE import SRBEModels
from utils.metrics import RegressionMetrics

In [4]:
exp_df = pd.read_csv('../../Data/Experimental/be_exp.csv', delimiter=',')


## Inference

In [5]:
sr_semf = SRBEModels(base_model="semf")
sr_dz = SRBEModels(base_model="dz")

In [6]:
exp_df["semf_pred"] = exp_df.apply(lambda x: semf_be(x["Z"], x["N"]), axis=1)
exp_df["dz_pred"] = exp_df.apply(lambda x: dz_be(x["Z"], x["N"]), axis=1)

exp_df["semf_sr_pred"] = exp_df.apply(lambda x: sr_semf(x["Z"], x["N"]), axis=1)
exp_df["dz_sr_pred"] = exp_df.apply(lambda x: sr_dz(x["Z"], x["N"]), axis=1)

# Regression Metrics

In [14]:
semf_metrics = RegressionMetrics(exp_df["BE"], exp_df["semf_pred"], "SEMF")()
dz_metrics = RegressionMetrics(exp_df["BE"], exp_df["dz_pred"], "DZ")()
semf_sr_metrics = RegressionMetrics(exp_df["BE"], exp_df["semf_sr_pred"], "SEMF_SR")()
dz_sr_metrics = RegressionMetrics(exp_df["BE"], exp_df["dz_sr_pred"], "DZ_SR")()
results = pd.concat([semf_metrics, dz_metrics, semf_sr_metrics, dz_sr_metrics], axis=0)
results.sort_values(by="RMSE", inplace=False)

,R2,MSE,RMSE,MAE,MAPE,RMSPE,Max Error,Explained Variance Score
DZ_SR,0.999997,0.728790,0.853692,0.640380,0.088993,0.190633,4.902942,0.999997
DZ,0.999992,1.580478,1.257171,0.822816,0.096232,0.185628,6.797186,0.999993
SEMF,0.999916,17.463550,4.178941,2.978731,0.401497,0.873391,20.136952,0.999918
SEMF_SR,0.999878,25.368226,5.036688,1.880518,0.346813,2.016229,108.699950,0.999878


#